# Sustainable Life Planner - Amazon Bedrock Version

This notebook demonstrates a multi-agent sustainable living planner using:
- **Amazon Bedrock Agents** for orchestration
- **Bedrock Knowledge Bases** for RAG
- **Claude Sonnet 4.5** as the foundation model

## Architecture
1. Knowledge Base Agent (RAG)
2. Carbon Footprint Calculator
3. Habit Planning Agent
4. Recommendation Engine

## Setup and Installation

In [2]:
# Install required packages
!pip install boto3 botocore -q

# Install python-dotenv only if not in Colab
try:
    import google.colab
    print("Running in Google Colab - skipping python-dotenv")
except ImportError:
    !pip install python-dotenv -q
    print("Running locally - installed python-dotenv")

In [4]:
import boto3
import json
import os
from datetime import datetime

# Google Colab setup - store these in Colab's secret manager:
# 1. Go to the key icon (🔑) in the left sidebar
# 2. Add secrets: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_REGION
try:
    from google.colab import userdata
    # Set AWS credentials from Colab secrets
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    AWS_REGION = userdata.get('AWS_REGION')
    KNOWLEDGE_BASE_ID = userdata.get('KNOWLEDGE_BASE_ID') if 'KNOWLEDGE_BASE_ID' in userdata._keys else 'C75LOPFVP2'
    print("✅ Using Google Colab secrets for AWS credentials")
except ImportError:
    # Fallback for local execution
    from dotenv import load_dotenv
    load_dotenv()
    AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')
    KNOWLEDGE_BASE_ID = os.getenv('KNOWLEDGE_BASE_ID', 'C75LOPFVP2')
    print("✅ Using local .env file for AWS credentials")

# AWS Configuration
AWS_REGION = AWS_REGION or 'us-east-1'
KNOWLEDGE_BASE_ID = KNOWLEDGE_BASE_ID or 'C75LOPFVP2'  # Set after creating KB

# Initialize Bedrock clients
bedrock_runtime = boto3.client('bedrock-runtime', region_name=AWS_REGION)
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=AWS_REGION)
bedrock_agent = boto3.client('bedrock-agent', region_name=AWS_REGION)

# Model ID for Claude Sonnet 4.5 (using inference profile)
MODEL_ID = 'us.anthropic.claude-sonnet-4-5-20250929-v1:0'

print(f"✅ Initialized Bedrock clients in {AWS_REGION}")
print(f"📚 Model: {MODEL_ID}")

✅ Initialized Bedrock clients in us-east-1
📚 Model: us.anthropic.claude-sonnet-4-5-20250929-v1:0


## Agent 1: Sustainability Knowledge Expert (with RAG)

In [5]:
def query_knowledge_base(query, knowledge_base_id):
    """Query Bedrock Knowledge Base for sustainability information"""
    if not knowledge_base_id:
        return "Knowledge Base not configured. Using fallback data."
    
    try:
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=knowledge_base_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {'numberOfResults': 5}
            }
        )
        
        results = []
        for result in response['retrievalResults']:
            results.append(result['content']['text'])
        
        return '\n\n'.join(results)
    except Exception as e:
        return f"Error querying KB: {str(e)}"

def knowledge_agent(user_profile, knowledge_base_id):
    """Agent 1: Retrieve sustainability knowledge"""
    query = f"""Sustainability information for:
    Transportation: {user_profile['transportation']}
    Diet: {user_profile['diet']}
    Energy: {user_profile['energy_usage']}"""
    
    kb_context = query_knowledge_base(query, knowledge_base_id)
    
    prompt = f"""You are a sustainability expert. Based on this knowledge base context:

{kb_context}

Summarize relevant sustainability information for:
- Transportation: {user_profile['transportation']}
- Diet: {user_profile['diet']}
- Energy usage: {user_profile['energy_usage']}

Provide key facts and emission factors."""
    
    response = bedrock_runtime.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

print("✅ Agent 1: Knowledge Expert ready")

✅ Agent 1: Knowledge Expert ready


## Agent 2: Carbon Footprint Analyzer

In [6]:
def carbon_footprint_agent(user_profile, knowledge_context):
    """Agent 2: Calculate carbon footprint"""
    prompt = f"""You are a carbon footprint analyst. Using this context:

{knowledge_context}

Calculate the annual carbon footprint for:
- Transportation: {user_profile['transportation']}
- Diet: {user_profile['diet']}
- Energy usage: {user_profile['energy_usage']}

Provide:
1. Breakdown by category (tons CO2/year)
2. Total annual emissions
3. Comparison to average

Use realistic emission factors."""
    
    response = bedrock_runtime.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

print("✅ Agent 2: Carbon Analyzer ready")

✅ Agent 2: Carbon Analyzer ready


## Agent 3: Sustainable Habit Coach

In [7]:
def habit_planning_agent(user_profile, carbon_analysis):
    """Agent 3: Create weekly sustainable habits"""
    prompt = f"""You are a sustainable habit coach. Based on this carbon footprint analysis:

{carbon_analysis}

User goals: {user_profile['goals']}

Create a 7-day sustainable routine with:
- Specific daily habits
- Actionable steps
- Focus on high-impact changes

Format as:
Monday: [habit]
Tuesday: [habit]
..."""
    
    response = bedrock_runtime.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

print("✅ Agent 3: Habit Coach ready")

✅ Agent 3: Habit Coach ready


## Agent 4: Eco-Friendly Recommendation Engine

In [8]:
def recommendation_agent(user_profile, carbon_analysis, habit_plan):
    """Agent 4: Generate personalized recommendations"""
    prompt = f"""You are an eco-friendly advisor. Based on:

Carbon Analysis:
{carbon_analysis}

Habit Plan:
{habit_plan}

User goals: {user_profile['goals']}

Provide top 5 prioritized recommendations:
1. [Recommendation] - Expected impact: [X tons CO2/year]
2. ...

Focus on practical, high-impact changes."""
    
    response = bedrock_runtime.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [{"role": "user", "content": prompt}]
        })
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

print("✅ Agent 4: Recommendation Engine ready")

✅ Agent 4: Recommendation Engine ready


## Orchestrator: Sequential Agent Execution

In [9]:
def create_sustainable_life_plan(user_profile, knowledge_base_id=None):
    """Orchestrate all agents sequentially"""
    print("🌱 Starting Sustainable Life Planner...\n")
    print("=" * 60)
    
    # Agent 1: Knowledge retrieval
    print("📚 Agent 1: Gathering sustainability knowledge...")
    knowledge = knowledge_agent(user_profile, knowledge_base_id or KNOWLEDGE_BASE_ID)
    print("✅ Complete\n")
    
    # Agent 2: Carbon calculation
    print("📊 Agent 2: Calculating carbon footprint...")
    carbon_analysis = carbon_footprint_agent(user_profile, knowledge)
    print("✅ Complete\n")
    
    # Agent 3: Habit planning
    print("📅 Agent 3: Creating weekly habits...")
    habit_plan = habit_planning_agent(user_profile, carbon_analysis)
    print("✅ Complete\n")
    
    # Agent 4: Recommendations
    print("💡 Agent 4: Generating recommendations...")
    recommendations = recommendation_agent(user_profile, carbon_analysis, habit_plan)
    print("✅ Complete\n")
    
    print("=" * 60)
    
    return {
        'knowledge': knowledge,
        'carbon_analysis': carbon_analysis,
        'habit_plan': habit_plan,
        'recommendations': recommendations
    }

print("✅ Orchestrator ready")

✅ Orchestrator ready


## Run the Planner

In [10]:
# Define user profile
user_profile = {
    'transportation': 'gasoline car, 20 km daily commute',
    'diet': 'meat-eating, occasional fast food',
    'energy_usage': 'standard home, no solar, AC usage high',
    'goals': 'reduce carbon footprint by 30% in 6 months'
}

print("User Profile:")
for key, value in user_profile.items():
    print(f"  {key}: {value}")
print()

User Profile:
  transportation: gasoline car, 20 km daily commute
  diet: meat-eating, occasional fast food
  energy_usage: standard home, no solar, AC usage high
  goals: reduce carbon footprint by 30% in 6 months



In [11]:
# Execute the planner
result = create_sustainable_life_plan(user_profile)

🌱 Starting Sustainable Life Planner...

📚 Agent 1: Gathering sustainability knowledge...
✅ Complete

📊 Agent 2: Calculating carbon footprint...
✅ Complete

📅 Agent 3: Creating weekly habits...
✅ Complete

💡 Agent 4: Generating recommendations...
✅ Complete



## Display Results

In [12]:
print("\n" + "=" * 60)
print("📋 YOUR SUSTAINABLE LIFE PLAN")
print("=" * 60)

print("\n📚 SUSTAINABILITY KNOWLEDGE")
print("-" * 60)
print(result['knowledge'])

print("\n\n📊 CARBON FOOTPRINT ANALYSIS")
print("-" * 60)
print(result['carbon_analysis'])

print("\n\n📅 WEEKLY SUSTAINABLE HABITS")
print("-" * 60)
print(result['habit_plan'])

print("\n\n💡 TOP RECOMMENDATIONS")
print("-" * 60)
print(result['recommendations'])

print("\n" + "=" * 60)


📋 YOUR SUSTAINABLE LIFE PLAN

📚 SUSTAINABILITY KNOWLEDGE
------------------------------------------------------------
# Personal Sustainability Assessment

## Current Carbon Footprint Profile

### Transportation Impact
- **Daily commute:** 20 km in a gasoline car
- **Emissions:** 0.19 kg CO2/km
- **Daily total:** 3.8 kg CO2 (20 km × 0.19)
- **Annual total:** ~1.4 tons CO2 (for commute alone)
- **Context:** Transportation accounts for 29% of personal emissions, with single-occupancy vehicles being the largest contributor

### Diet Impact
- **Current:** Meat-eating diet with occasional fast food
- **Annual emissions:** 2.5 tons CO2/year
- **Context:** Food production accounts for 15% of personal carbon footprint

### Home Energy Impact
- **Current setup:** No solar panels, high AC usage, standard appliances
- **Context:** Home energy contributes 20% of personal carbon footprint
- **Key concern:** High AC usage significantly increases electricity consumption

## Estimated Total Annual Fo

## Optional: Setup Bedrock Knowledge Base

To enable RAG with the knowledge base, follow these steps:

### 1. Create S3 Bucket and Upload Data

In [ ]:
# This cell helps you upload the knowledge base to S3
s3_client = boto3.client('s3', region_name=AWS_REGION)
BUCKET_NAME = f'sustainable-planner-kb-{datetime.now().strftime("%Y%m%d%H%M%S")}'

# Create bucket
try:
    if AWS_REGION == 'us-east-1':
        s3_client.create_bucket(Bucket=BUCKET_NAME)
    else:
        s3_client.create_bucket(
            Bucket=BUCKET_NAME,
            CreateBucketConfiguration={'LocationConstraint': AWS_REGION}
        )
    print(f"✅ Created S3 bucket: {BUCKET_NAME}")
except Exception as e:
    print(f"Error: {e}")

# Upload knowledge base file
try:
    s3_client.upload_file(
        '../knowledge_base/sustainability_guide.txt',
        BUCKET_NAME,
        'sustainability_guide.txt'
    )
    print(f"✅ Uploaded knowledge base to s3://{BUCKET_NAME}/sustainability_guide.txt")
except Exception as e:
    print(f"Error uploading file: {e}")

### 2. Create Knowledge Base Programmatically

In [ ]:
import time

iam_client = boto3.client('iam', region_name=AWS_REGION)
aoss_client = boto3.client('opensearchserverless', region_name=AWS_REGION)

KB_NAME = 'sustainable-planner-kb'
KB_ROLE_NAME = f'{KB_NAME}-role'
COLLECTION_NAME = 'sustainable-kb-vec'  # Max 32 chars

print("🔧 Creating Knowledge Base infrastructure...\n")

# 1. Create IAM role for Knowledge Base
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "bedrock.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

try:
    role_response = iam_client.create_role(
        RoleName=KB_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy)
    )
    role_arn = role_response['Role']['Arn']
    print(f"✅ Created IAM role: {role_arn}")
    
    # Attach policies
    iam_client.attach_role_policy(
        RoleName=KB_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess'
    )
    
    policy_doc = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Action": ["s3:GetObject", "s3:ListBucket"],
            "Resource": [f"arn:aws:s3:::{BUCKET_NAME}/*", f"arn:aws:s3:::{BUCKET_NAME}"]
        }]
    }
    
    iam_client.put_role_policy(
        RoleName=KB_ROLE_NAME,
        PolicyName='S3Access',
        PolicyDocument=json.dumps(policy_doc)
    )
    
    time.sleep(10)  # Wait for IAM propagation
    
except iam_client.exceptions.EntityAlreadyExistsException:
    role_arn = iam_client.get_role(RoleName=KB_ROLE_NAME)['Role']['Arn']
    print(f"ℹ️  Using existing role: {role_arn}")

# 2. Create OpenSearch Serverless collection
try:
    collection_response = aoss_client.create_collection(
        name=COLLECTION_NAME,
        type='VECTORSEARCH'
    )
    print(f"✅ Creating OpenSearch collection: {COLLECTION_NAME}")
    print("   (This may take 3-5 minutes...)")
    
    # Wait for collection to be active
    while True:
        status = aoss_client.batch_get_collection(
            names=[COLLECTION_NAME]
        )['collectionDetails'][0]['status']
        
        if status == 'ACTIVE':
            break
        print(f"   Status: {status}...")
        time.sleep(30)
    
    collection_arn = aoss_client.batch_get_collection(
        names=[COLLECTION_NAME]
    )['collectionDetails'][0]['arn']
    
    print(f"✅ Collection active: {collection_arn}")
    
except aoss_client.exceptions.ConflictException:
    collection_arn = aoss_client.batch_get_collection(
        names=[COLLECTION_NAME]
    )['collectionDetails'][0]['arn']
    print(f"ℹ️  Using existing collection: {collection_arn}")

# 3. Create Knowledge Base
try:
    kb_response = bedrock_agent.create_knowledge_base(
        name=KB_NAME,
        roleArn=role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': f'arn:aws:bedrock:{AWS_REGION}::foundation-model/amazon.titan-embed-text-v1'
            }
        },
        storageConfiguration={
            'type': 'OPENSEARCH_SERVERLESS',
            'opensearchServerlessConfiguration': {
                'collectionArn': collection_arn,
                'vectorIndexName': 'bedrock-knowledge-base-index',
                'fieldMapping': {
                    'vectorField': 'bedrock-knowledge-base-default-vector',
                    'textField': 'AMAZON_BEDROCK_TEXT_CHUNK',
                    'metadataField': 'AMAZON_BEDROCK_METADATA'
                }
            }
        }
    )
    
    kb_id = kb_response['knowledgeBase']['knowledgeBaseId']
    print(f"\n✅ Created Knowledge Base: {kb_id}")
    
except Exception as e:
    if 'already exists' in str(e).lower():
        kbs = bedrock_agent.list_knowledge_bases()['knowledgeBaseSummaries']
        kb_id = next(kb['knowledgeBaseId'] for kb in kbs if kb['name'] == KB_NAME)
        print(f"\nℹ️  Using existing Knowledge Base: {kb_id}")
    else:
        raise e

# 4. Create data source
try:
    ds_response = bedrock_agent.create_data_source(
        knowledgeBaseId=kb_id,
        name='s3-sustainability-docs',
        dataSourceConfiguration={
            'type': 'S3',
            's3Configuration': {
                'bucketArn': f'arn:aws:s3:::{BUCKET_NAME}'
            }
        }
    )
    
    ds_id = ds_response['dataSource']['dataSourceId']
    print(f"✅ Created data source: {ds_id}")
    
except Exception as e:
    if 'already exists' in str(e).lower():
        sources = bedrock_agent.list_data_sources(knowledgeBaseId=kb_id)['dataSourceSummaries']
        ds_id = sources[0]['dataSourceId']
        print(f"ℹ️  Using existing data source: {ds_id}")
    else:
        raise e

# 5. Start ingestion job
print("\n🔄 Starting data ingestion...")
ingestion_response = bedrock_agent.start_ingestion_job(
    knowledgeBaseId=kb_id,
    dataSourceId=ds_id
)

job_id = ingestion_response['ingestionJob']['ingestionJobId']

# Wait for ingestion
while True:
    job_status = bedrock_agent.get_ingestion_job(
        knowledgeBaseId=kb_id,
        dataSourceId=ds_id,
        ingestionJobId=job_id
    )['ingestionJob']['status']
    
    if job_status == 'COMPLETE':
        print("✅ Ingestion complete!")
        break
    elif job_status == 'FAILED':
        print("❌ Ingestion failed")
        break
    
    print(f"   Status: {job_status}...")
    time.sleep(10)

print(f"\n" + "="*60)
print(f"🎉 Knowledge Base Setup Complete!")
print(f"="*60)
print(f"Knowledge Base ID: {kb_id}")
print(f"\n💡 Update your .env file with:")
print(f"   KNOWLEDGE_BASE_ID={kb_id}")
print(f"\n🔄 Or run the planner with:")
print(f"   result = create_sustainable_life_plan(user_profile, '{kb_id}')")

### 3. Test Knowledge Base

In [ ]:
# Test KB retrieval
test_query = "What are the carbon emissions for gasoline cars?"
print(f"Query: {test_query}\n")

test_response = bedrock_agent_runtime.retrieve(
    knowledgeBaseId=kb_id,
    retrievalQuery={'text': test_query},
    retrievalConfiguration={
        'vectorSearchConfiguration': {'numberOfResults': 3}
    }
)

print("Retrieved Results:\n")
for i, result in enumerate(test_response['retrievalResults'], 1):
    print(f"{i}. {result['content']['text'][:200]}...\n")

### 4. Run Planner with Knowledge Base

In [ ]:
# Now run with KB enabled
result_with_kb = create_sustainable_life_plan(user_profile, kb_id)